In [ ]:

from whale.data_io.data_loader import WhaleDataModule
import torch
from pytorch_lightning import seed_everything
import numpy  as np
from matplotlib import pyplot as plt
seed_everything(1234)

whale_dm = WhaleDataModule(
    data_dir="/network/projects/aia/whale_call/LABELS/FWC_1CH_mini", batch_size=1
)
whale_dm.setup()



In [ ]:
train_ds = whale_dm.train_ds
train_size = len(train_ds)
idx_rands = np.random.choice(train_size, 9)


In [ ]:
from whale.models import UNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt_path = '/network/projects/aia/whale_call/mlruns/ckpts/unet'
model = UNet.load_from_checkpoint(ckpt_path+'/epoch=0-step=63.ckpt')
model.eval();
model.to(device);

In [ ]:
fig,axs = plt.subplots(3,3,figsize=(9,9))
from matplotlib import pyplot as plt
for i in range(9):
    idx_rand = idx_rands[i]
    row = i // 3
    col = i%3
    data_sel = train_ds[idx_rand]
    sig = data_sel['sig']
    pred = torch.argmax(model(sig.unsqueeze(0).to(device)),axis=1)
    target = data_sel['target']
    
    axs[row][col].plot(np.arange(501),sig[0]);
    axs[row][col].plot(np.arange(501),pred[0].cpu(),color='red');
    axs[row][col].plot(np.arange(501),target);

